In [2]:
import numpy as np
import bempp.api

def coulomb_potential(q, p, Q, xq, E):
    """
    Computes the electrostatic potential due to a point monopole, dipole, 
    quadrupole distribution at the position of the multipoles.
    See equation 29 of amoeba bem document. 
    Inputs:
    ------- 
        q: array size N with charges
        p: array size (Nx3) with dipoles
        Q: array size (Nx3x3) with quadrupoles
        xq: array size (Nx3) with positions of multipoles
        E: dielectric constant
    Returns:
    -------
        phi: electrostatic potential at the position of the multipoles
    """

    phi = np.zeros(len(xq))
    T2  = np.zeros((len(xq),3,3))
    for i in range(len(xq)):
        Ri = xq[i]-xq
        Rnorm = np.sqrt(np.sum(Ri*Ri, axis=1))

        for j in np.where(Rnorm>1e-10)[0]: #remove singularity
            T0 = 1/Rnorm[j]
            T1 = Ri[j,:]/Rnorm[j]**3
            T2[j,:,:] = np.ones((3,3))*Ri[j,:]*np.transpose(np.ones((3,3))*Ri[j,:])/Rnorm[j]**5

            phi[i] += q[j]*T0 + np.sum(T1[:]*p[j,:]) + 0.5*np.sum(np.sum(T2[j,:,:]*Q[j,:,:],axis=1),axis=0)

    phi /= (4*np.pi*E)

    return phi

def coulomb_potential_thole(p, alpha, xq, E):
    """
    Computes the electrostatic potential due to a point dipole 
    distribution at the position of the multipoles.
    See equation 29 of amoeba bem document. 
    Uses Thole damping
    Inputs:
    ------- 
        p: array size (Nx3) with polarizable dipoles
        alpha: array size (Nx3x3) with polarizabilities (tensor)
        xq: array size (Nx3) with positions of multipoles
        E: dielectric constant
    Returns:
    -------
        phi: electrostatic potential at the position of the multipoles
    """

    phi = np.zeros(len(xq))
    T2  = np.zeros((len(xq),3,3))
    for i in range(len(xq)):
        Ri = xq[i]-xq
        Rnorm = np.sqrt(np.sum(Ri*Ri, axis=1))

        for j in np.where(Rnorm>1e-10)[0]: #remove singularity

            # Thole damping for polarizable dipoles (valid for thole factor=1)
            damp = (alpha[i,0,0]*alpha[j,0,0])**(0.16666667)
            damp += 1e-12
            damp = -(Rnorm[j]/damp)**3
            expdamp = np.exp(damp)
            scale3 = 1 - expdamp
            scale5 = 1 - expdamp*(1-damp)

            T1 = Ri[j,:]/Rnorm[j]**3 * scale3

            phi[i] += np.sum(T1[:]*p[j,:]) 

    phi /= (4*np.pi*E)

    return phi

def coulomb_field(q, p, Q, xq, E):
    """
    Computes the electric field due to a point monopole, dipole, quadrupole distribution
    at the position of the multipoles. The field is defined as E=-nabla*phi.
    See equation 52 of kirkwood multipole, and Equation 30 of amoeba bem document.
    Inputs:
    ------- 
        q: array size N with charges
        p: array size (Nx3) with dipoles
        Q: array size (Nx3x3) with quadrupoles
        xq: array size (Nx3) with positions of multipoles
        E: dielectric constant
    Returns:
    -------
        Efield: electric field at the position of the multipoles
    """
    Efield = np.zeros((len(xq),3))
    T0 = np.zeros((len(xq),3))
    T1 = np.zeros((len(xq),3,3))
    T2 = np.zeros((len(xq),3,3,3))
    for i in range(len(xq)):
        Ri = xq[i]-xq
        Rnorm = np.sqrt(np.sum(Ri*Ri, axis=1))


        for j in np.where(Rnorm>1e-10)[0]: #remove singularity

            T0[j,:]   = -Ri[j,:]/Rnorm[j]**3
            T1[j,:,:] = np.identity(3)/Rnorm[j]**3 - 3*np.ones((3,3))*Ri[j,:]*np.transpose(np.ones((3,3))*Ri[j,:])/Rnorm[j]**5

            # the ordering in aux will be k,j,i looking at Eq 52 of kirkwood multipole
            aux = np.zeros((3,3,3))
            for k in range(3):
                aux[k,:,:] = np.ones((3,3))*Ri[j,:]*np.transpose(np.ones((3,3))*Ri[j,:])*Ri[j,k]
            aux *= -5/Rnorm[j]**7

            for k in range(3):
                aux[:,:,k] += np.identity(3)*Ri[j,k]/Rnorm[j]**5
            for k in range(3):
                aux[:,k,:] += np.identity(3)*Ri[j,k]/Rnorm[j]**5

            T2[j,:,:,:] = aux

            for k in range(3):
                Efield[i,k] += T0[j,k]*q[j] + np.sum(T1[j,k,:]*p[j,:]) + 0.5*np.sum(np.sum(T2[j,k,:,:]*Q[j,:,:],axis=1),axis=0)

    Efield /= -(4*np.pi*E)
    return Efield

def coulomb_field_thole(q, p, Q, alpha, xq, E):
    """
    Computes the electric field due to a point monpole, dipole and quadrupole
    at the position of the multipoles. The field is defined as E=-nabla*phi.
    Uses Thole damping.
    See equation 52 of kirkwood multipole, and Equation 30 of amoeba bem document.
    Inputs:
    ------- 
        q: array size N with charges
        p: array size (Nx3) with dipoles
        Q: array size (Nx3x3) with quadrupoles
        alpha: array size (Nx3x3) with polarizabilities
        xq: array size (Nx3) with positions of multipoles
        E: dielectric constant
    Returns:
    -------
        Efield: electric field at the position of the multipoles
    """
    Efield = np.zeros((len(xq),3))
    T0 = np.zeros((len(xq),3))
    T1 = np.zeros((len(xq),3,3))
    T2 = np.zeros((len(xq),3,3,3))
    for i in range(len(xq)):
        Ri = xq[i]-xq
        Rnorm = np.sqrt(np.sum(Ri*Ri, axis=1))


        for j in np.where(Rnorm>1e-10)[0]: #remove singularity

            # Thole damping for polarizable dipoles (valid for thole factor=1)
            damp = (alpha[i,0,0]*alpha[j,0,0])**(0.16666667)
            damp += 1e-12
            damp = -(Rnorm[j]/damp)**3
            expdamp = np.exp(damp)
            scale3 = 1 - expdamp
            scale5 = 1 - expdamp*(1-damp)
            scale7 = 1 - expdamp*(1-damp+0.6*damp*damp) 

            T0[j,:]   = -Ri[j,:]/Rnorm[j]**3 * scale3
            T1[j,:,:] = np.identity(3)/Rnorm[j]**3*scale3 - 3*np.ones((3,3))*Ri[j,:]*np.transpose(np.ones((3,3))*Ri[j,:])/Rnorm[j]**5*scale5

            # the ordering in aux will be k,j,i looking at Eq 52 of kirkwood multipole
            aux = np.zeros((3,3,3))
            for k in range(3):
                aux[k,:,:] = np.ones((3,3))*Ri[j,:]*np.transpose(np.ones((3,3))*Ri[j,:])*Ri[j,k]
            aux *= -5/Rnorm[j]**7*scale7

            for k in range(3):
                aux[:,:,k] += np.identity(3)*Ri[j,k]/Rnorm[j]**5*scale5
            for k in range(3):
                aux[:,k,:] += np.identity(3)*Ri[j,k]/Rnorm[j]**5*scale5

            T2[j,:,:,:] = aux

            for k in range(3):
                Efield[i,k] += T0[j,k]*q[j] + np.sum(T1[j,k,:]*p[j,:]) + 0.5*np.sum(np.sum(T2[j,k,:,:]*Q[j,:,:],axis=1),axis=0) 

    Efield /= -(4*np.pi*E)
    return Efield

def coulomb_ddpotential(q, p, Q, xq, E):
    """
    Computes the second derivative of the electrostatic potential due 
    to a point monopole, dipole, and quadrupole distribution at the 
    position of the multipoles. See equation 29 and 43 of amoeba bem document. 
    Inputs:
    ------- 
        q: array size N with charges
        p: array size (Nx3) with dipoles
        Q: array size (Nx3x3) with quadrupoles
        xq: array size (Nx3) with positions of multipoles
        E: dielectric constant
    Returns:
    -------
        ddphi: second derivative of electrostatic potential at the 
                position of the multipoles
    """
    ddphi = np.zeros((len(xq),3,3))
    T0 = np.zeros((len(xq),3,3))
    T1 = np.zeros((len(xq),3,3,3))
    T2 = np.zeros((len(xq),3,3,3,3))
    for i in range(len(xq)):
        Ri = xq[i]-xq
        Rnorm = np.sqrt(np.sum(Ri*Ri, axis=1))

        for j in np.where(Rnorm>1e-10)[0]: #remove singularity
            T0[j,:,:] = -np.identity(3)/Rnorm[j]**3 + 3*np.ones((3,3))*Ri[j,:]*np.transpose(np.ones((3,3))*Ri[j,:])/Rnorm[j]**5

            # the ordering in aux will be k,j,i looking at Eq 52 of kirkwood multipole
            aux = np.zeros((3,3,3))
            for k in range(3):
                aux[k,:,:] = np.ones((3,3))*Ri[j,:]*np.transpose(np.ones((3,3))*Ri[j,:])*Ri[j,k]
            aux *= 15/Rnorm[j]**7

            for k in range(3):
                aux[:,:,k] -= 3*np.identity(3)*Ri[j,k]/Rnorm[j]**5
                aux[:,k,:] -= 3*np.identity(3)*Ri[j,k]/Rnorm[j]**5
                aux[k,:,:] -= 3*np.identity(3)*Ri[j,k]/Rnorm[j]**5

            T1[j,:,:,:] = aux

            for k in range(3):
                for l in range(3):
                    for m in range(3):
                        for n in range(3):
                            dkl = (k==l)*1.0
                            dkm = (k==m)*1.0
                            dkn = (k==n)*1.0
                            dlm = (l==m)*1.0
                            dln = (l==n)*1.0

                            T2[j,k,l,m,n] = -7*Ri[j,k]*Ri[j,l]*Ri[j,m]*Ri[j,n]/Rnorm[j]**2  \
                                           + Ri[j,m]*Ri[j,n]*dkl + Ri[j,l]*Ri[j,n]*dkm      \
                                           + Ri[j,m]*Ri[j,l]*dkn + Ri[j,k]*Ri[j,n]*dlm      \
                                           + Ri[j,m]*Ri[j,k]*dln                            \
                                           - (dkm*dln + dlm*dkn)*Rnorm[j]**2/5
            T2 *= -5/Rnorm[j]**7

            for k in range(3):
                for l in range(3):
                    ddphi[i,k,l] += T0[j,k,l]*q[j] + np.sum(T1[j,k,l,:]*p[j,:]) + 0.5*np.sum(np.sum(T2[j,k,l,:,:]*Q[j,:,:],axis=1),axis=0)
    
    ddphi /= (4*np.pi*E)

    return ddphi

def coulomb_ddpotential_thole(p, alpha, xq, E):
    """
    Computes the second derivative of the electrostatic potential due 
    to a point dipole distribution at the 
    position of the multipoles. 
    Uses Thole damping.
    See equation 29 and 43 of amoeba bem document. 
    Inputs:
    ------- 
        p: array size (Nx3) with dipoles
        alpha: array size (Nx3x3) with polarizabilities
        xq: array size (Nx3) with positions of multipoles
        E: dielectric constant
    Returns:
    -------
        ddphi: second derivative of electrostatic potential at the 
                position of the multipoles
    """
    ddphi = np.zeros((len(xq),3,3))
    T1 = np.zeros((len(xq),3,3,3))
    for i in range(len(xq)):
        Ri = xq[i]-xq
        Rnorm = np.sqrt(np.sum(Ri*Ri, axis=1))

        for j in np.where(Rnorm>1e-10)[0]: #remove singularity

            # Thole damping for polarizable dipoles (valid for thole factor=1)
            damp = (alpha[i,0,0]*alpha[j,0,0])**(0.16666667)
            damp += 1e-12
            damp = -(Rnorm[j]/damp)**3
            expdamp = np.exp(damp)
            scale3 = 1 - expdamp
            scale5 = 1 - expdamp*(1-damp)
            scale7 = 1 - expdamp*(1-damp+0.6*damp*damp) 

            # the ordering in aux will be k,j,i looking at Eq 52 of kirkwood multipole
            aux = np.zeros((3,3,3))
            for k in range(3):
                aux[k,:,:] = np.ones((3,3))*Ri[j,:]*np.transpose(np.ones((3,3))*Ri[j,:])*Ri[j,k]
            aux *= 15/Rnorm[j]**7 * scale7

            for k in range(3):
                aux[:,:,k] -= 3*np.identity(3)*Ri[j,k]/Rnorm[j]**5 * scale5
                aux[:,k,:] -= 3*np.identity(3)*Ri[j,k]/Rnorm[j]**5 * scale5
                aux[k,:,:] -= 3*np.identity(3)*Ri[j,k]/Rnorm[j]**5 * scale5

            T1[j,:,:,:] = aux

            for k in range(3):
                for l in range(3):
                    ddphi[i,k,l] += np.sum(T1[j,k,l,:]*p[j,:]) 
    
    ddphi /= (4*np.pi*E)

    return ddphi

def coulomb_energy_multipole(q, p_per, p_pol, Q, alpha, xq, E):
    """
    Computes the Coulomb energy from a collection of point
    multipoles, according to equation 38 of amoeba bem document.
    
    Inputs:
    ------ 
        q : array size N with charges of multipoles
        p_per : array size (Nx3) with permanent dipoles of multipoles
        p_pol : array size (Nx3) with polarizable dipoles of multipoles
        Q : array size (Nx3x3) with quadrupoles of multipoles
        xq: array size Nx3 with positions of multipoles
    p_perm: array size (Nx3) with permanent dipoles of multipoles
        E : float, dielectric constant
    Outputs:
    -------
        E_coul: (float) free energy  
    """
    qe = 1.60217646e-19
    Na = 6.0221415e23
    E_0 = 8.854187818e-12
    cal2J = 4.184 

    phi   = coulomb_potential(q, p_per, Q, xq, E)
    dphi  = -1*coulomb_field(q, p_per, Q, xq, E)
    ddphi = coulomb_ddpotential(q, p_per, Q, xq, E)

    dummy1 = np.zeros((len(q)))        # dummy charges
    dummy2 = np.zeros((len(q),3,3))    # dummy quadrupoles

    phi += coulomb_potential_thole(p_pol, alpha, xq, E)
    dphi += -1*coulomb_field_thole(dummy1, p_pol, dummy2, alpha, xq, E)
    ddphi += coulomb_ddpotential_thole(p_pol, alpha, xq, E)

    cons = qe**2*Na*1e-3*1e10/(cal2J*E_0)
    E_coul = 0.5*cons*(np.sum(q*phi) + np.sum(np.sum(p_per*dphi,axis=1)) + np.sum(np.sum(np.sum(Q*ddphi,axis=2),axis=1))/6)

    return E_coul

def coulomb_polarizable_dipole(q, p_per, Q, alpha, xq, E):
    """
    Computes polarized dipole component of a collection of polarizabe multipoles
    Used in eq 56 of kirkwood multipole
    Inputs:
    ------
        q: array size N with charges of multipoles
        p_per: array size (Nx3) with permanent dipoles of multipoles
        Q: array size (Nx3x3) with quadrupoles of multipoles
        alpha: array size (Nqx3x3) with polarizability of dipoles (considered as a tensor)
        xq: array size Nx3 with positions of multipoles
        E : float, dielectric constant
    Returns:
    -------
        p_pol: array size (Nx3) with polarizable component of dipoles
        Efield: array size (Nx3) with electrostatic field that polarized the multipoles 
    """
    p_pol      = np.zeros((len(xq),3))
    dipole_diff= 1. 
    p_pol_prev = np.ones((len(xq),3))

    iteration = 0
    SOR = 0.7
    while dipole_diff>1e-6:
        iteration += 1
        p_tot = p_per + p_pol
    
        Efield = coulomb_field_thole(q, p_tot, Q, alpha, xq, E)
        
        for k in range(len(q)):
            p_pol[k] = p_pol[k]*(1-SOR) + np.dot(alpha[k],4*np.pi*Efield[k])*SOR # 4*pi because alpha in Tinker
                                                                           # comes in atomic units that
                                                                           # that already include the 1/4pi

        dipole_diff = np.sqrt(np.sum((np.linalg.norm(p_pol_prev-p_pol,axis=1))**2)/len(p_pol))
        p_pol_prev = p_pol.copy()

    print('Took %i iterations for vacuum induced dipole to converge'%iteration)
    return p_pol, Efield

def solvation_energy(q, p, Q, phi, dphi, ddphi):
    solv_energy = 2*np.pi*332.064*(np.sum(q*phi) + np.sum(p*dphi) + np.sum(Q*ddphi)/6)
    return solv_energy

def solvent_potential_first_derivate(xq, h, neumann_space, dirichl_space, solution_neumann, solution_dirichl):
    """
    Compute the first derivate of potential due to solvent
    in the position of the points
    Inputs:
    -------
        xq: Array size (Nx3) whit positions to calculate the derivate.
        h: Float number, distance for the central difference.
        neumann_space: Bempp Function space
        dirichl_space: Bempp Function space
        solution_neumann: Data of dphi in boundary
        solution_dirichl: Data of phi in boundary

    Return:

    dpdr: Derivate of the potential in the points positions.
    """

    dpdr = np.zeros([len(xq), 3])
    dist = np.array(([h,0,0],[0,h,0],[0,0,h]))
    # x axis derivate
    dx = xq[:] + dist[0]
    dx = np.concatenate((dx, xq[:] - dist[0]))
    slpo = bempp.api.operators.potential.laplace.single_layer(neumann_space, dx.transpose())
    dlpo = bempp.api.operators.potential.laplace.double_layer(dirichl_space, dx.transpose())
    phi = slpo.evaluate(solution_neumann) - dlpo.evaluate(solution_dirichl)
    dpdx = 0.5*(phi[0,:len(xq)] - phi[0,len(xq):])/h
    dpdr[:,0] = dpdx

    #y axis derivate
    dy = xq[:] + dist[1]
    dy = np.concatenate((dy, xq[:] - dist[1]))
    slpo = bempp.api.operators.potential.laplace.single_layer(neumann_space, dy.transpose())
    dlpo = bempp.api.operators.potential.laplace.double_layer(dirichl_space, dy.transpose())
    phi = slpo.evaluate(solution_neumann) - dlpo.evaluate(solution_dirichl)
    dpdy = 0.5*(phi[0,:len(xq)] - phi[0,len(xq):])/h
    dpdr[:,1] = dpdy

    #z axis derivate
    dz = xq[:] + dist[2]
    dz = np.concatenate((dz, xq[:] - dist[2]))
    slpo = bempp.api.operators.potential.laplace.single_layer(neumann_space, dz.transpose())
    dlpo = bempp.api.operators.potential.laplace.double_layer(dirichl_space, dz.transpose())
    phi = slpo.evaluate(solution_neumann) - dlpo.evaluate(solution_dirichl)
    dpdz = 0.5*(phi[0,:len(xq)] - phi[0,len(xq):])/h
    dpdr[:,2] = dpdz

    return dpdr

def solvent_potential_second_derivate(x_q, h, neumann_space, dirichl_space, solution_neumann, solution_dirichl):
    """
    Compute the second derivate of potential due to solvent
    in the position of the points

    xq: Array size (Nx3) whit positions to calculate the derivate.
    h: Float number, distance for the central difference.

    Return:

    ddphi: Second derivate of the potential in the positions of points.
    """
    ddphi = np.zeros((len(x_q),3,3))
    dist = np.array(([h,0,0],[0,h,0],[0,0,h]))
    for i in range(3):
        for j in np.where(np.array([0, 1, 2]) >= i)[0]:
            if i==j:
                dp = np.concatenate((x_q[:] + dist[i], x_q[:], x_q[:] - dist[i]))
                slpo = bempp.api.operators.potential.laplace.single_layer(neumann_space, dp.transpose())
                dlpo = bempp.api.operators.potential.laplace.double_layer(dirichl_space, dp.transpose())
                phi = slpo.evaluate(solution_neumann) - dlpo.evaluate(solution_dirichl)
                ddphi[:,i,j] = (phi[0,:len(x_q)] - 2*phi[0,len(x_q):2*len(x_q)] + phi[0, 2*len(x_q):])/(h**2)
      
            else:
                dp = np.concatenate((x_q[:] + dist[i] + dist[j], x_q[:] - dist[i] - dist[j], x_q[:] + dist[i] - dist[j], x_q[:] - dist[i] + dist[j]))
                slpo = bempp.api.operators.potential.laplace.single_layer(neumann_space, dp.transpose())
                dlpo = bempp.api.operators.potential.laplace.double_layer(dirichl_space, dp.transpose())
                phi = slpo.evaluate(solution_neumann) - dlpo.evaluate(solution_dirichl)
                ddphi[:,i,j] = (phi[0,:len(x_q)] + phi[0,len(x_q):2*len(x_q)] - phi[0, 2*len(x_q):3*len(x_q)] - phi[0, 3*len(x_q):])/(4*h**2)
                ddphi[:,j,i] = (phi[0,:len(x_q)] + phi[0,len(x_q):2*len(x_q)] - phi[0, 2*len(x_q):3*len(x_q)] - phi[0, 3*len(x_q):])/(4*h**2)
  
    return ddphi

In [3]:
q = np.array([1,-1,-1]) #Corresponde a un arreglo de N elementos, con la magnitud de las cargas puntuales.
d = np.array([[0.,1.,0.],[1.,0.,0.],[0.,0.,-1.]]) #Arreglo de Nx3 elementos, corresponde a los vectores de los dipolos.
Q = np.array([[[1.,0.,0.],[0.,-1.,0.],[0.,0.,0.]],[[0.,0.,0.],[0.,1.,0.],[0.,0.,-1.]],[[1.,0.,0.],[0.,0.,0.],[0.,0.,-1.]]])
alpha = np.array([[[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]],[[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]],[[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]]])

x_q = np.array([[1e-12,1e-12,1e-12],[1.,1.414213562,1.],[-1.,-1.,1.414213562]]) #Ubicación de los multipolos.

bohr = 0.52917721067
d *= bohr
Q *= 2*bohr**2

In [4]:
ep_in = 1
ep_ex = 80
k = 0.125

In [8]:
import bempp.api
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
import inspect
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import gmres
def iteration_counter(x):
    global array_it, array_frame, it_count
    it_count += 1
    frame = inspect.currentframe().f_back
    array_it = np.append(array_it, it_count)
    array_frame = np.append(array_frame, frame.f_locals["resid"])
    print(F"Gmres iteration {it_count}, residual: {x}")

def multipole_calculations_quadrupoles(q, d, Q, x_q, ep_in, ep_ex, k, alpha, h, radius=3, element_size=0.21, maxiter=100, 
                                       gmres_maxiter=500, mu=np.zeros((len(x_q),3)), tol=1e-5, gmres_tol=1e-5, SOR=0.7):
    global array_it, array_frame, it_count, p
  
    grid = bempp.api.shapes.sphere(r= radius, h= element_size)
    dirichl_space = bempp.api.function_space(grid, "DP", 0)
    neumann_space = bempp.api.function_space(grid, "DP", 0)

    x_b = np.zeros((2*dirichl_space.global_dof_count))

    identity = sparse.identity(dirichl_space, dirichl_space, dirichl_space); #Identidad
    slp_in   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space); #Single Layer potential, interior
    dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space); #Double Layer potential, interior
    slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, k); #Single Layer potential, exterior
    dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, k); #Double Layer potential, exterior

    blocked = bempp.api.BlockedOperator(2, 2);
    blocked[0, 0] = 0.5*identity + dlp_in;
    blocked[0, 1] = -slp_in;
    blocked[1, 0] = 0.5*identity - dlp_out;
    blocked[1, 1] = (ep_in/ep_ex)*slp_out;
    lhs = blocked.strong_form(); #Matriz con los operadores, lado izquierdo de la ecuación.

    dummy1 = np.zeros((len(q)))
    dummy2 = np.zeros((len(q), 3, 3))

    E_mult_perm = coulomb_field(q, d, Q, x_q, ep_in)


    for iter_number in range(maxiter):

        print(F"------- Iteration {iter_number +1} -------")
    
        array_it, array_frame, it_count = np.array([]), np.array([]), 0

        p = d + mu #d is the permanent dipole, mu is the polarizable dipole

        @bempp.api.real_callable
        def multipolar_quadrupoles_charges_fun(x, n, i, result):
            T2 = np.zeros((len(x_q),3,3))
            dist = x - x_q
            norm = np.sqrt(np.sum((dist*dist), axis = 1))
            T0 = 1/norm[:]
            T1 = np.transpose(dist.transpose()/norm**3)
            T2[:,:,:] = np.ones((len(x_q),3,3))[:]*dist.reshape((len(x_q),1,3))*np.transpose(np.ones((len(x_q),3,3))*dist.reshape((len(x_q),1,3)), (0,2,1))/norm.reshape((len(x_q),1,1))**5
            phi = np.sum(q[:]*T0[:]) + np.sum(T1[:]*p[:]) + 0.5*np.sum(np.sum(T2[:]*Q[:],axis=1))
            result[0] = phi/(4*np.pi*ep_in)

        charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun = multipolar_quadrupoles_charges_fun)
        rhs = np.concatenate([charged_grid_fun.coefficients, np.zeros(neumann_space.global_dof_count)])
      
        x, info = gmres(lhs, rhs, x0=x_b, callback=iteration_counter, callback_type="pr_norm", tol=gmres_tol, maxiter=gmres_maxiter, restart = 1000)

        x_b = x.copy()

        solution_dirichl = bempp.api.GridFunction(dirichl_space, coefficients=x[:dirichl_space.global_dof_count])
        solution_neumann = bempp.api.GridFunction(neumann_space, coefficients=x[dirichl_space.global_dof_count:])

        # Calculo de potencial y su derivada debido al solvente.

        slpo = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
        dlpo = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
        phi_solvent = slpo.evaluate(solution_neumann) - dlpo.evaluate(solution_dirichl)

        dphi_solvent = solvent_potential_first_derivate(x_q, h, neumann_space, dirichl_space, solution_neumann, solution_dirichl)
        ddphi_solvent = solvent_potential_second_derivate(x_q, h, neumann_space, dirichl_space, solution_neumann, solution_dirichl)

        #Calculo del campo electrico debido a los multipolos.

        E_mult_pol = coulomb_field_thole(dummy1, mu, dummy2, alpha, x_q, ep_in)
        E_mult = E_mult_perm + E_mult_pol

        E_total = dphi_solvent*-1 + E_mult

        #Comprobar convergencia.

        mu_b = mu.copy()
        for i in range(len(x_q)):
            mu[i] = mu[i]*(1-SOR) + np.dot(alpha[i], E_total[i]*4*np.pi)*SOR

        dipole_diff = np.max(np.sqrt(np.sum((mu-mu_b)**2, axis = 1)))
        if dipole_diff<tol:
            print(F"The result has converged in {iter_number+1} iterations")
            break

    G_diss_solv = solvation_energy(q, d, Q, phi_solvent, dphi_solvent, ddphi_solvent)
    print(F"Solvent contributión: {G_diss_solv}")
    G_diss_mult = coulomb_energy_multipole(q, d, mu, Q, alpha, x_q, ep_in)
    print(F"Multipoles contributión: {G_diss_mult}")

    #Calculo de G_vacc

    p_pol_vac, Epol_vac = coulomb_polarizable_dipole(q, d, Q, alpha, x_q, ep_in)
    p_vac_tot = d + p_pol_vac
    G_vacc = coulomb_energy_multipole(q, d, p_pol_vac, Q, alpha, x_q, ep_in)
    print(F"Coulomb vacuum energy: {G_vacc}")

    total_energy = G_diss_solv + G_diss_mult - G_vacc
    print(F"Para este caso, la energía libre de solvatación es de {total_energy} [kcal/Mol]")

    return total_energy

In [9]:
p_pol_vac, Epol_vac = coulomb_polarizable_dipole(q, d, Q, alpha, x_q, ep_in)
solv_energy = multipole_calculations_quadrupoles(q, d, Q, x_q, ep_in, ep_ex, k, alpha, 0.01, mu=p_pol_vac, tol=1e-3, 
                                                 gmres_tol=1e-5, element_size=0.15, maxiter=1000, gmres_maxiter=999)

Took 19 iterations for vacuum induced dipole to converge


/home/felipevicencio/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


------- Iteration 1 -------
Gmres iteration 1, residual: 0.9943209918726752
Gmres iteration 2, residual: 0.43557521059169496
Gmres iteration 3, residual: 0.3838381479685464
Gmres iteration 4, residual: 0.18600219326011708
Gmres iteration 5, residual: 0.14808289682489403
Gmres iteration 6, residual: 0.08170584294230337
Gmres iteration 7, residual: 0.06123827276065618
Gmres iteration 8, residual: 0.03389283060059513
Gmres iteration 9, residual: 0.022894068485146434
Gmres iteration 10, residual: 0.014177636987161386
Gmres iteration 11, residual: 0.008995893866639822
Gmres iteration 12, residual: 0.005673656137741091
Gmres iteration 13, residual: 0.00329079104383057
Gmres iteration 14, residual: 0.0022511885246106113
Gmres iteration 15, residual: 0.0012250313485780676
Gmres iteration 16, residual: 0.0008706764025629977
Gmres iteration 17, residual: 0.0005078907165180084
Gmres iteration 18, residual: 0.0004314740665361975
Gmres iteration 19, residual: 0.0003781320180792269
Gmres iteration 2

Gmres iteration 19, residual: 6.6880030217847566e-06
------- Iteration 8 -------
Gmres iteration 1, residual: 0.004915939529966776
Gmres iteration 2, residual: 0.002465524713507732
Gmres iteration 3, residual: 0.0020219623600486424
Gmres iteration 4, residual: 0.001087817823054363
Gmres iteration 5, residual: 0.0008350635381719315
Gmres iteration 6, residual: 0.00042606147236497964
Gmres iteration 7, residual: 0.0002942455891502122
Gmres iteration 8, residual: 0.0001711156266412875
Gmres iteration 9, residual: 0.00014299120598919814
Gmres iteration 10, residual: 0.000130379455273027
Gmres iteration 11, residual: 0.00010571867701992135
Gmres iteration 12, residual: 6.564886065575575e-05
Gmres iteration 13, residual: 3.809329845244593e-05
Gmres iteration 14, residual: 2.514552583722321e-05
Gmres iteration 15, residual: 1.3810218294771258e-05
Gmres iteration 16, residual: 9.900946792066255e-06
------- Iteration 9 -------
Gmres iteration 1, residual: 0.003374799991597484
Gmres iteration 2,